In [ ]:
#|hide
#|default_exp processors

# nbprocess.processors
- Some processors for `NBProcessor`

In [ ]:
#|export
from nbprocess.read import *
from nbprocess.imports import *
from nbprocess.process import *

from fastcore.imports import *
from fastcore.xtras import *

In [ ]:
from fastcore.test import *

In [ ]:
#|export
def process_outputs(f, nbp, cell, args=None):
    "Helper function to use from a processor to process outputs in `cell`"
    for outp in getattr(cell, 'outputs', []): f(outp, args)

In [ ]:
_test_file = '../tests/docs_test.ipynb'

def _run_procs(procs=None, preprocs=None, postprocs=None):
    nbp = NBProcessor(_test_file, procs, preprocs=preprocs, postprocs=postprocs)
    nbp.process()
    return '\n'.join([str(cell) for cell in nbp.nb.cells])

In [ ]:
#|export
def meta_(nbp, cell, key, *args): cell.metadata[key] = args

In [ ]:
#|export
def show_meta(cell):
    "Show cell metadata"
    if cell.metadata: print(cell.metadata)

To inject metadata make a comment in a cell with the following pattern: `#cell_meta:{key=value}`. Note that `#meta` is an alias for `#cell_meta`

For example, at the moment, this notebook has no cells with metadata, which we can see b using `show_meta`:

In [ ]:
_run_procs([show_meta]);

However, after we process this notebook with `inject_meta`, the appropriate metadata will be injected:

In [ ]:
_run_procs([meta_,show_meta]);

{'show_steps': ('start', 'train')}


In [ ]:
#| export
def insert_warning(nb):
    "Insert Autogenerated Warning Into Notebook after the first cell."
    content = "<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->"
    nb.cells.insert(1, AttrDict(cell_type='markdown', metadata={}, source=content))

This preprocessor inserts a warning in the markdown destination that the file is autogenerated.  This warning is inserted in the second cell so we do not interfere with front matter.

In [ ]:
res = _run_procs(preprocs=[insert_warning])
assert "<!-- WARNING: THIS FILE WAS AUTOGENERATED!" in res

In [ ]:
#|export
def remove_(nbp, cell, *args):
    for arg in args:
        if arg=='input': cell['source'] = ''
        elif arg=='output': del(cell['outputs'])
        elif arg=='cell': del(cell['source'])
        else: raise NameError(arg)

In [ ]:
res = _run_procs(remove_)
assert 'you will not be able to see this cell at all either' not in res
assert 'THE output is removed' not in res and 'output is removed' in res
assert "the','code" not in res and 'code that created me' in res

In [ ]:
#|export
_re_hideline = re.compile(r'#\|\s*hide_line\s*$', re.MULTILINE)
def hide_line(cell):
    "Hide lines of code in code cells with the directive `hide_line` at the end of a line of code"
    if cell.cell_type == 'code' and _re_hideline.search(cell.source):
        cell.source = '\n'.join([c for c in cell.source.splitlines() if not _re_hideline.search(c)])

In [ ]:
res = _run_procs(hide_line)
assert r"def show():\n    a = 2" in res

In [ ]:
#|export
def _filter_stream(outp, args):
    if not args or outp.output_type != 'stream': return
    outp['text'] = [l for l in outp.text if not re.search('|'.join(args), l)]

def filter_stream_(nbp, cell, *words):
    "Remove output lines containing any of `words` in `cell` stream output"
    process_outputs(_filter_stream, nbp, cell, words)

In [ ]:
res = _run_procs(filter_stream_)
exp=r"'A line\n', 'Another line.\n'"
assert exp in res

In [ ]:
#|export
_magics_pattern = re.compile(r'^\s*(%%|%).*', re.MULTILINE)

def clean_magics(cell):
    "A preprocessor to remove cell magic commands"
    if cell.cell_type == 'code': cell.source = _magics_pattern.sub('', cell.source).strip()

In [ ]:
res = _run_procs(clean_magics)
assert "%%" not in res

In [ ]:
#|export
_bash_pattern = re.compile('^\s*!', flags=re.MULTILINE)

def bash_identify(cell):
    "A preprocessor to identify bash commands and mark them appropriately"
    if cell.cell_type == 'code' and _bash_pattern.search(cell.source):
        cell.metadata.magics_language = 'bash'
        cell['source'] = _bash_pattern.sub('', cell.source).strip()

When we issue a shell command in a notebook with `!`, we need to change the code-fence from `python` to `bash` and remove the `!`:

In [ ]:
res = _run_procs(bash_identify)
assert "'echo" in res

In [ ]:
#export
_re_hdr_dash = re.compile(r'^#+\s+.*\s+-\s*$', re.MULTILINE)

def rm_header_dash(cell):
    "Remove headings that end with a dash -"
    src = cell.source.strip()
    if cell.cell_type == 'markdown' and src.startswith('#') and src.endswith(' -'): del(cell['source'])

In [ ]:
res = _run_procs(rm_header_dash)
assert 'some words' in res
assert 'A heading to Hide' not in res and 'Another Heading' not in res and 'Yet another heading to hide' not in res

In [ ]:
#export
def rm_export(cell):
    "Remove cells that are exported or hidden"
    if any(o in cell.directives_ for o in ('export','exporti','hide','default_exp')): del(cell['source'])

In [ ]:
res = _run_procs(rm_export)
assert 'dontshow' not in res

In [ ]:
#export
_re_showdoc = re.compile(r'^ShowDoc', re.MULTILINE)
def _is_showdoc(cell): return cell['cell_type'] == 'code' and _re_showdoc.search(cell.source)

def clean_show_doc(cell):
    "Remove ShowDoc input cells"
    if not _isShowDoc(cell): return
    cell.source = ''

In [ ]:
res = _run_procs(clean_show_doc)
assert 'ShowDoc(1)' not in res
assert 'showdoc output' in res

## Export -

In [ ]:
#|skip
from nbprocess.export import nbs_export
nbs_export()